In [61]:
import pandas as pd
import numpy as np

# Corrected paths with raw strings or double backslashes
menu_item_df = pd.read_csv(r'C:\Users\Soumya Nanda\NYPL-menus\MenuItem.csv')
dish_df = pd.read_csv(r'C:\Users\Soumya Nanda\NYPL-menus\OpenRefine\NYPL-Dish.csv')
menu_df = pd.read_csv(r'C:\Users\Soumya Nanda\NYPL-menus\OpenRefine\NYPL-Menu.csv')
menu_page_df = pd.read_csv(r'C:\Users\Soumya Nanda\NYPL-menus\OpenRefine\NYPL-MenuPage.csv')


In [62]:
valid_menu_ids = menu_df['id'].unique()
menu_page_df = menu_page_df[menu_page_df['menu_id'].isin(valid_menu_ids)]

menu_page_df.to_csv(r'C:\Users\Soumya Nanda\NYPL-menus\OpenRefine\NYPL-MenuPage.csv', index=False)

In [63]:
# Step 1: Ensure that menu_page_id and dish_id are not null
menu_item_df = menu_item_df.dropna(subset=['menu_page_id', 'dish_id'])

# Step 2: Ensure that menu_page_id and dish_id exist in their respective tables
menu_item_df = menu_item_df[menu_item_df['menu_page_id'].isin(menu_page_df['id'])]
menu_item_df = menu_item_df[menu_item_df['dish_id'].isin(dish_df['id'])]

# Step 3: Ensure price and high_price are numeric and non-negative
menu_item_df['price'] = pd.to_numeric(menu_item_df['price'], errors='coerce').fillna(0)
menu_item_df['high_price'] = pd.to_numeric(menu_item_df['high_price'], errors='coerce').fillna(menu_item_df['price'])
menu_item_df = menu_item_df[(menu_item_df['price'] >= 0) & (menu_item_df['high_price'] >= 0)]

# Step 4: Check for duplicates and remove them
menu_item_df = menu_item_df.drop_duplicates()


In [64]:
# Step 5: Remove rows in dish_df with invalid 'first_appeared' and 'last_appeared'
dish_df = dish_df[(dish_df['first_appeared'] >= 1851) & (dish_df['last_appeared'] <= 2024)]

# Step 6: Merge menu_item_df with menu_page_df to get menu_id
menu_item_df = pd.merge(menu_item_df, menu_page_df[['id', 'menu_id']], left_on='menu_page_id', right_on='id', how='left', suffixes=('', '_page'))
menu_item_df = menu_item_df.rename(columns={'menu_id': 'menu_id_from_page'})

# Step 7: Merge the result with menu_df to get the date
menu_item_df = pd.merge(menu_item_df, menu_df[['id', 'date']], left_on='menu_id_from_page', right_on='id', how='left', suffixes=('', '_menu'))
menu_item_df = menu_item_df.rename(columns={'date': 'menu_date'})

# Step 8: Extract the year from the date and convert it to numeric
menu_item_df['date'] = menu_item_df['menu_date'].str[:4].astype(float)

# Step 9: Remove rows with NaN values in the date and keep dates within 1851 to 2024
menu_item_df = menu_item_df.dropna(subset=['date'])
menu_item_df = menu_item_df[(menu_item_df['date'] >= 1851) & (menu_item_df['date'] <= 2024)]

# Step 10: Ensure menu_item_df only contains dish_id values that exist in dish_df
valid_dish_ids = dish_df['id'].unique()
menu_item_df = menu_item_df[menu_item_df['dish_id'].isin(valid_dish_ids)]

# Step 11: Remove rows where price and high_price values are zero
menu_item_df = menu_item_df[(menu_item_df['price'] > 0) & (menu_item_df['high_price'] > 0)]

# Step 12: Add a column for dish_name using dish_id and id from dish_df
menu_item_df = pd.merge(menu_item_df, dish_df[['id', 'name']], left_on='dish_id', right_on='id', how='left', suffixes=('', '_dish'))
menu_item_df = menu_item_df.rename(columns={'name': 'dish_name'})

In [65]:
# Display the cleaned DataFrames
print("Cleaned Dish DataFrame:")
print(dish_df.head())
print("\nCleaned MenuItem DataFrame with Date (Year):")
print(menu_item_df.head())

# Save the cleaned DataFrames
dish_df.to_csv(r'C:\Users\Soumya Nanda\NYPL-menus\OpenRefine\NYPL-Dish.csv', index=False)
menu_item_df.to_csv(r'C:\Users\Soumya Nanda\NYPL-menus\OpenRefine\NYPL-MenuItem.csv', index=False)

Cleaned Dish DataFrame:
   id                    name  description  menus_appeared  times_appeared  \
0   2           Chicken gumbo          NaN             111             117   
1   3     Tomato aux croutons          NaN              13              13   
2   4         Onion au gratin          NaN              41              41   
3   5             St. Emilion          NaN              66              68   
4   8  Chicken soup with rice          NaN              48              49   

   first_appeared  last_appeared  lowest_price  highest_price  
0            1895           1960          0.10            0.8  
1            1893           1917          0.25            0.4  
2            1900           1971          0.25            1.0  
3            1881           1981          0.00           18.0  
4            1897           1961          0.10            0.6  

Cleaned MenuItem DataFrame with Date (Year):
   id  menu_page_id  price  high_price  dish_id               created_at  \
0